In [9]:
library(httr)
library(jsonlite)

## functions

> search()    
  >> imdb_search()   

> get_runtime()    
> get_rating()      
> get_genre()     
> read_plot()      
> get_actors()    
> get_writer()     
> get_director()    
> get_awards()    
> get_poster()     

> get_top10_recommendations()
> surprise_me()


In [10]:
# this is not user-facing function. 
# This is internal function used in for loops of lists of movies.

imdb_search <- function(imdb_id){
  
    id_search_url <- "http://www.omdbapi.com/?i="
    apikey <- 'apikey=45abab5f'

    id_url <- paste0(id_search_url, paste(imdb_id, apikey, sep = '&'))
    movie <- httr::GET(url = id_url)
    movie_httr <- httr::content(movie, as = "text")
    movie_json <- fromJSON(movie_httr)
    if (movie_json$Response == 'True'){
        return(movie_json)
    } else {
        print(movie_json$Error)
        return (NULL)
    }   
}

In [3]:
imdb_search('tt2015381')

,Source,Value
,<chr>,<chr>
1,Internet Movie Database,8.0/10
2,Rotten Tomatoes,91%
3,Metacritic,76/100


In [18]:
imdb_search('tt20151')

[1] "Incorrect IMDb ID."


NULL

In [20]:
typeof(imdb_search('tt2015381'))

[1] "list"

In [11]:
# get runtime of movies by search

get_runtime <- function(search_text, type = ""){
    
    # type can be : movie, series, episode, game
   
    base_search_url <- "http://www.omdbapi.com/?s="
    type = paste0("type=",type)
    apikey <- "apikey=a79b2c95"
    search <- gsub(" ", "+", search_text)

    search_url <- paste0(base_search_url, paste(search, type, apikey, sep = '&'))
    result <- httr::GET(url = search_url)
    result_httr <- httr::content(result, as = "text")
    result_json <- fromJSON(result_httr)
    
    title <- NA
    type <- NA
    year <- NA
    runtime <- NA
    
    for (i in 1:length(result_json$Search$imdbID)){
        imdb_id = result_json$Search$imdbID[i]
        imdb_id_details <- imdb_search(imdb_id)
        title[i] <- imdb_id_details$Title
        type[i] <- imdb_id_details$Type
        year[i] <- imdb_id_details$Year
        runtime[i] <- imdb_id_details$Runtime
    }
    
    output = data.frame(title, type, suppressWarnings(as.numeric(year)), suppressWarnings(as.numeric(sapply(strsplit(runtime, " "), "[[", 1))))
    names(output)[1] <- "Title"
    names(output)[2] <- "Type"
    names(output)[3] <- "Year"
    names(output)[4] <- "Runtime (in minutes)"
    return (output)
}


In [9]:
get_runtime("guardians")

Title,Type,Year,Runtime (in minutes)
<chr>,<chr>,<dbl>,<dbl>
Guardians of the Galaxy,movie,2014,121
Guardians of the Galaxy Vol. 2,movie,2017,136
Rise of the Guardians,movie,2012,97
Rise of the Guardians,movie,2012,97
Legend of the Guardians: The Owls of Ga'Hoole,movie,2010,97
Guardians,movie,2017,89
7 Guardians of the Tomb,movie,2018,90
Naruto the Movie 3: Guardians of the Crescent Moon Kingdom,movie,2006,95
Guardians of the Galaxy,series,NA,22


In [12]:
colnames(get_runtime("guardians"))[1] 

[1] "Title"

In [17]:
get_runtime("guardians")[[1]]

[1] "Guardians of the Galaxy"                                   
 [2] "Guardians of the Galaxy Vol. 2"                            
 [3] "Rise of the Guardians"                                     
 [4] "Rise of the Guardians"                                     
 [5] "Legend of the Guardians: The Owls of Ga'Hoole"             
 [6] "Guardians"                                                 
 [7] "7 Guardians of the Tomb"                                   
 [8] "Naruto the Movie 3: Guardians of the Crescent Moon Kingdom"
 [9] "Guardians of the Galaxy"                                   
[10] "Halo 5: Guardians"

In [6]:
# get imdb Rating of movies by search

get_rating <- function(search_text, type = ""){
    
    # type can be : movie, series, episode, game
   
    base_search_url <- "http://www.omdbapi.com/?s="
    type = paste0("type=",type)
    apikey <- "apikey=a79b2c95"
    search <- gsub(" ", "+", search_text)

    search_url <- paste0(base_search_url, paste(search, type, apikey, sep = '&'))
    result <- httr::GET(url = search_url)
    result_httr <- httr::content(result, as = "text")
    result_json <- fromJSON(result_httr)
    
    title <- NA
    type <- NA
    year <- NA
    rating <- NA
    
    for (i in 1:length(result_json$Search$imdbID)){
        imdb_id = result_json$Search$imdbID[i]
        imdb_id_details <- imdb_search(imdb_id)
        title[i] <- imdb_id_details$Title
        type[i] <- imdb_id_details$Type
        year[i] <- imdb_id_details$Year
        rating[i] <- imdb_id_details$imdbRating
    }
    
    output = data.frame(title, type, suppressWarnings(as.numeric(year)), suppressWarnings(as.numeric(rating)))
    names(output)[1] <- "Title"
    names(output)[2] <- "Type"
    names(output)[3] <- "Year"
    names(output)[4] <- "IMDB Rating"
    return (output)
}


In [7]:
get_rating("guardians")

Title,Type,Year,IMDB Rating
<chr>,<chr>,<dbl>,<dbl>
Guardians of the Galaxy,movie,2014,8.0
Guardians of the Galaxy Vol. 2,movie,2017,7.6
Rise of the Guardians,movie,2012,7.3
Rise of the Guardians,movie,2012,7.3
Legend of the Guardians: The Owls of Ga'Hoole,movie,2010,6.9
Guardians,movie,2017,3.9
7 Guardians of the Tomb,movie,2018,4.5
Naruto the Movie 3: Guardians of the Crescent Moon Kingdom,movie,2006,6.5
Guardians of the Galaxy,series,NA,7.4


In [8]:
# get genre of movies by search

get_genre <- function(search_text, type = ""){
    
    # type can be : movie, series, episode, game
   
    base_search_url <- "http://www.omdbapi.com/?s="
    type = paste0("type=",type)
    apikey <- "apikey=a79b2c95"
    search <- gsub(" ", "+", search_text)

    search_url <- paste0(base_search_url, paste(search, type, apikey, sep = '&'))
    result <- httr::GET(url = search_url)
    result_httr <- httr::content(result, as = "text")
    result_json <- fromJSON(result_httr)
    
    title <- NA
    type <- NA
    year <- NA
    genre <- NA
    
    for (i in 1:length(result_json$Search$imdbID)){
        imdb_id = result_json$Search$imdbID[i]
        imdb_id_details <- imdb_search(imdb_id)
        title[i] <- imdb_id_details$Title
        type[i] <- imdb_id_details$Type
        year[i] <- imdb_id_details$Year
        genre[i] <- imdb_id_details$Genre
    }
    
    output = data.frame(title, type, suppressWarnings(as.numeric(year)), genre)
    names(output)[1] <- "Title"
    names(output)[2] <- "Type"
    names(output)[3] <- "Year"
    names(output)[4] <- "Genre"
    return (output)
}


In [9]:
get_genre("guardians")

Title,Type,Year,Genre
<chr>,<chr>,<dbl>,<chr>
Guardians of the Galaxy,movie,2014,"Action, Adventure, Comedy, Sci-Fi"
Guardians of the Galaxy Vol. 2,movie,2017,"Action, Adventure, Comedy, Sci-Fi"
Rise of the Guardians,movie,2012,"Animation, Action, Adventure, Comedy, Family, Fantasy, Sci-Fi"
Rise of the Guardians,movie,2012,"Animation, Action, Adventure, Comedy, Family, Fantasy, Sci-Fi"
Legend of the Guardians: The Owls of Ga'Hoole,movie,2010,"Animation, Action, Adventure, Family, Fantasy, Mystery, Thriller, War"
Guardians,movie,2017,"Action, Adventure, Comedy, Fantasy, Sci-Fi, Thriller"
7 Guardians of the Tomb,movie,2018,"Action, Adventure, Horror"
Naruto the Movie 3: Guardians of the Crescent Moon Kingdom,movie,2006,"Animation, Action, Adventure"
Guardians of the Galaxy,series,NA,"Animation, Action, Adventure, Comedy, Drama, Family, Sci-Fi"


In [10]:
# read the plot of movies/games/series by search

read_plot <- function(search_text, type = "", plot = 'full'){
    
    # type can be : movie, series, episode, game
    # plot type can be : short, full
   
    base_search_url <- "http://www.omdbapi.com/?s="
    type = paste0("type=",type)
    plot = paste0("plot=",plot)
    apikey <- "apikey=a79b2c95"
    search <- gsub(" ", "+", search_text)

    search_url <- paste0(base_search_url, paste(search, type, plot, apikey, sep = '&'))
    result <- httr::GET(url = search_url)
    result_httr <- httr::content(result, as = "text")
    result_json <- fromJSON(result_httr)
    
    title <- NA
    type <- NA
    year <- NA
    plot <- NA
    
    for (i in 1:length(result_json$Search$imdbID)){
        imdb_id = result_json$Search$imdbID[i]
        imdb_id_details <- imdb_search(imdb_id)
        title[i] <- imdb_id_details$Title
        type[i] <- imdb_id_details$Type
        year[i] <- imdb_id_details$Year
        plot[i] <- imdb_id_details$Plot
    }
    
    output = data.frame(title, type, suppressWarnings(as.numeric(year)), plot)
    names(output)[1] <- "Title"
    names(output)[2] <- "Type"
    names(output)[3] <- "Year"
    names(output)[4] <- "Plot"
    return (output)
}


In [11]:
read_plot("guardians")

Title,Type,Year,Plot
<chr>,<chr>,<dbl>,<chr>
Guardians of the Galaxy,movie,2014,A group of intergalactic criminals must pull together to stop a fanatical warrior with plans to purge the universe.
Guardians of the Galaxy Vol. 2,movie,2017,"The Guardians struggle to keep together as a team while dealing with their personal family issues, notably Star-Lord's encounter with his father the ambitious celestial being Ego."
Rise of the Guardians,movie,2012,"When the evil spirit Pitch launches an assault on Earth, the Immortal Guardians team up to protect the innocence of children all around the world."
Rise of the Guardians,movie,2012,"When the evil spirit Pitch launches an assault on Earth, the Immortal Guardians team up to protect the innocence of children all around the world."
Legend of the Guardians: The Owls of Ga'Hoole,movie,2010,"When a young owl is abducted by an evil Owl army, he must escape with new-found friends and seek the legendary Guardians to stop the menace."
Guardians,movie,2017,"During the Cold War, an organization called ""Patriot"" created a super-hero squad, which includes members of multiple Soviet republics. For years, the heroes had to hide their identities, but in hard times they must show themselves again."
7 Guardians of the Tomb,movie,2018,"An innocent discovery of a well-preserved mummified Chinese Emperor from 200 B.C. unearths a two-thousand-year-old nightmare, a secret that should have remained buried."
Naruto the Movie 3: Guardians of the Crescent Moon Kingdom,movie,2006,"Naruto, Kakashi, Sakura, and Rock Lee are assigned to protect Michiru, the prince of the Land of the Moon, while he travel the world. The prince purchased a tiger Sham and the whole circus, which became the team's protection."
Guardians of the Galaxy,series,NA,The adventures of a band of space warriors who work to protect the universe from the evil overlord Thanos.


In [12]:
# get actors from movies by search

get_actors <- function(search_text, type = ""){
    
    # type can be : movie, series, episode, game
   
    base_search_url <- "http://www.omdbapi.com/?s="
    type = paste0("type=",type)
    apikey <- "apikey=a79b2c95"
    search <- gsub(" ", "+", search_text)

    search_url <- paste0(base_search_url, paste(search, type, apikey, sep = '&'))
    result <- httr::GET(url = search_url)
    result_httr <- httr::content(result, as = "text")
    result_json <- fromJSON(result_httr)
    
    title <- NA
    type <- NA
    year <- NA
    actors <- NA
    
    for (i in 1:length(result_json$Search$imdbID)){
        imdb_id = result_json$Search$imdbID[i]
        imdb_id_details <- imdb_search(imdb_id)
        title[i] <- imdb_id_details$Title
        type[i] <- imdb_id_details$Type
        year[i] <- imdb_id_details$Year
        actors[i] <- imdb_id_details$Actors
    }
    
    output = data.frame(title, type, suppressWarnings(as.numeric(year)), actors)
    names(output)[1] <- "Title"
    names(output)[2] <- "Type"
    names(output)[3] <- "Year"
    names(output)[4] <- "Actors"
    return (output)
}


In [13]:
get_actors("guardians galaxy")

Title,Type,Year,Actors
<chr>,<chr>,<dbl>,<chr>
Guardians of the Galaxy,movie,2014,"Chris Pratt, Zoe Saldana, Dave Bautista, Vin Diesel"
Guardians of the Galaxy Vol. 2,movie,2017,"Chris Pratt, Zoe Saldana, Dave Bautista, Vin Diesel"
Guardians of the Galaxy,series,NA,"Kevin Michael Richardson, Vanessa Marshall, David Sobolov, Will Friedle"
Guardians of the Galaxy: Inferno,movie,2017,"David Hasselhoff, Chris Pratt, Dave Bautista, Karen Gillan"
LEGO Marvel Super Heroes - Guardians of the Galaxy: The Thanos Threat,movie,2017,"Jonathan Adams, Trevor Devall, Will Friedle, Jennifer Hale"
Disneyland Resort: Guardians of the Galaxy - Mission Breakout!,movie,2017,"Bradley Cooper, Chris Pratt, Zoe Saldana, Dave Bautista"
Guardians of the Galaxy: The Telltale Series,game,2017,"Scott Porter, Brandon Paul Eells, Emily O'Brien, Adam Harrington"
Bonus Round: The Making of 'Guardians of the Galaxy Vol. 2',movie,2017,"Kevin Feige, James Gunn, Chris Pratt"
Guardians of the Galaxy Vol. 3,movie,2023,"Elizabeth Debicki, Karen Gillan, Zoe Saldana, Chris Pratt"


In [14]:
# get writer from movies by search

get_writer <- function(search_text, type = ""){
    
    # type can be : movie, series, episode, game
   
    base_search_url <- "http://www.omdbapi.com/?s="
    type = paste0("type=",type)
    apikey <- "apikey=a79b2c95"
    search <- gsub(" ", "+", search_text)

    search_url <- paste0(base_search_url, paste(search, type, apikey, sep = '&'))
    result <- httr::GET(url = search_url)
    result_httr <- httr::content(result, as = "text")
    result_json <- fromJSON(result_httr)
    
    title <- NA
    type <- NA
    year <- NA
    writer <- NA
    
    for (i in 1:length(result_json$Search$imdbID)){
        imdb_id = result_json$Search$imdbID[i]
        imdb_id_details <- imdb_search(imdb_id)
        title[i] <- imdb_id_details$Title
        type[i] <- imdb_id_details$Type
        year[i] <- imdb_id_details$Year
        writer[i] <- imdb_id_details$Writer
    }
    
    output = data.frame(title, type, suppressWarnings(as.numeric(year)), writer)
    names(output)[1] <- "Title"
    names(output)[2] <- "Type"
    names(output)[3] <- "Year"
    names(output)[4] <- "Writer"
    return (output)
}


In [15]:
get_writer("guardians galaxy")

Title,Type,Year,Writer
<chr>,<chr>,<dbl>,<chr>
Guardians of the Galaxy,movie,2014,"James Gunn, Nicole Perlman, Dan Abnett (based on the Marvel comics by), Andy Lanning (based on the Marvel comics by), Bill Mantlo (character created by: Rocket Raccoon), Keith Giffen (character created by: Rocket Raccoon), Jim Starlin (characters created by: Drax the Destroyer, Gamora & Thanos), Steve Englehart (character created by: Star-Lord), Steve Gan (character created by: Star-Lord), Steve Gerber (character created by: Howard the Duck), Val Mayerik (character created by: Howard the Duck)"
Guardians of the Galaxy Vol. 2,movie,2017,"James Gunn, Dan Abnett (based on the Marvel comics by), Andy Lanning (based on the Marvel comics by), Steve Englehart (Star-Lord created by), Steve Gan (Star-Lord created by), Jim Starlin (Gamora and Drax created by), Stan Lee (Groot created by), Larry Lieber (Groot created by), Jack Kirby (Groot created by), Bill Mantlo (Rocket Raccoon created by), Keith Giffen (Rocket Raccoon created by), Steve Gerber (Howard the Duck created by), Val Mayerik (Howard the Duck created by)"
Guardians of the Galaxy,series,NA,"Dan Abnett, Andy Lanning, Marty Isenberg, Henry Gilroy"
Guardians of the Galaxy: Inferno,movie,2017,James Gunn
LEGO Marvel Super Heroes - Guardians of the Galaxy: The Thanos Threat,movie,2017,"Mark Hoffmeier, Dan Abnett (based on the marvel comics by), Andy Lanning (based on the marvel comics by), Bill Mantlo (character created by: Rocket Raccoon), Keith Giffen (character created by: Rocket Raccoon), Steve Englehart (character created by: Star-Lord), Steve Gan (character created by: Star-Lord), Jim Starlin (characters created by: Thanos, Gamora & Drax)"
Disneyland Resort: Guardians of the Galaxy - Mission Breakout!,movie,2017,N/A
Guardians of the Galaxy: The Telltale Series,game,2017,"Emily Grace Buck (Story by, segment ""Under Pressure""), Erica Harnell (additional writing), Erica Harrell, Zack Keller, Nicole Martinez, Meghan Thornton, Tim Williams"
Bonus Round: The Making of 'Guardians of the Galaxy Vol. 2',movie,2017,N/A
Guardians of the Galaxy Vol. 3,movie,2023,"James Gunn, Dan Abnett (comic book), Andy Lanning (comic book)"


In [16]:
# get director from movies by search

get_director <- function(search_text, type = ""){
    
    # type can be : movie, series, episode, game
   
    base_search_url <- "http://www.omdbapi.com/?s="
    type = paste0("type=",type)
    apikey <- "apikey=a79b2c95"
    search <- gsub(" ", "+", search_text)

    search_url <- paste0(base_search_url, paste(search, type, apikey, sep = '&'))
    result <- httr::GET(url = search_url)
    result_httr <- httr::content(result, as = "text")
    result_json <- fromJSON(result_httr)
    
    title <- NA
    type <- NA
    year <- NA
    director <- NA
    
    for (i in 1:length(result_json$Search$imdbID)){
        imdb_id = result_json$Search$imdbID[i]
        imdb_id_details <- imdb_search(imdb_id)
        title[i] <- imdb_id_details$Title
        type[i] <- imdb_id_details$Type
        year[i] <- imdb_id_details$Year
        director[i] <- imdb_id_details$Director
    }
    
    output = data.frame(title, type, suppressWarnings(as.numeric(year)), director)
    names(output)[1] <- "Title"
    names(output)[2] <- "Type"
    names(output)[3] <- "Year"
    names(output)[4] <- "Director"
    return (output)
}


In [17]:
get_director("guardians galaxy")

Title,Type,Year,Director
<chr>,<chr>,<dbl>,<chr>
Guardians of the Galaxy,movie,2014,James Gunn
Guardians of the Galaxy Vol. 2,movie,2017,James Gunn
Guardians of the Galaxy,series,NA,N/A
Guardians of the Galaxy: Inferno,movie,2017,"James Gunn, David Yarovesky"
LEGO Marvel Super Heroes - Guardians of the Galaxy: The Thanos Threat,movie,2017,Michael D. Black
Disneyland Resort: Guardians of the Galaxy - Mission Breakout!,movie,2017,N/A
Guardians of the Galaxy: The Telltale Series,game,2017,"Jonathan Stauder, Sean Ainsworth"
Bonus Round: The Making of 'Guardians of the Galaxy Vol. 2',movie,2017,N/A
Guardians of the Galaxy Vol. 3,movie,2023,James Gunn


In [18]:
# get awards from movies by search

get_awards <- function(search_text, type = ""){
    
    # type can be : movie, series, episode, game
   
    base_search_url <- "http://www.omdbapi.com/?s="
    type = paste0("type=",type)
    apikey <- "apikey=a79b2c95"
    search <- gsub(" ", "+", search_text)

    search_url <- paste0(base_search_url, paste(search, type, apikey, sep = '&'))
    result <- httr::GET(url = search_url)
    result_httr <- httr::content(result, as = "text")
    result_json <- fromJSON(result_httr)
    
    title <- NA
    type <- NA
    year <- NA
    awards <- NA
    
    for (i in 1:length(result_json$Search$imdbID)){
        imdb_id = result_json$Search$imdbID[i]
        imdb_id_details <- imdb_search(imdb_id)
        title[i] <- imdb_id_details$Title
        type[i] <- imdb_id_details$Type
        year[i] <- imdb_id_details$Year
        awards[i] <- imdb_id_details$Awards
    }
    
    output = data.frame(title, type, suppressWarnings(as.numeric(year)), awards)
    names(output)[1] <- "Title"
    names(output)[2] <- "Type"
    names(output)[3] <- "Year"
    names(output)[4] <- "Awards"
    return (output)
}


In [19]:
get_awards("guardians galaxy")

Title,Type,Year,Awards
<chr>,<chr>,<dbl>,<chr>
Guardians of the Galaxy,movie,2014,Nominated for 2 Oscars. Another 52 wins & 100 nominations.
Guardians of the Galaxy Vol. 2,movie,2017,Nominated for 1 Oscar. Another 15 wins & 57 nominations.
Guardians of the Galaxy,series,NA,2 wins & 2 nominations.
Guardians of the Galaxy: Inferno,movie,2017,N/A
LEGO Marvel Super Heroes - Guardians of the Galaxy: The Thanos Threat,movie,2017,N/A
Disneyland Resort: Guardians of the Galaxy - Mission Breakout!,movie,2017,1 nomination.
Guardians of the Galaxy: The Telltale Series,game,2017,N/A
Bonus Round: The Making of 'Guardians of the Galaxy Vol. 2',movie,2017,N/A
Guardians of the Galaxy Vol. 3,movie,2023,N/A


In [22]:
# get link to the poster from movies by search

get_poster <- function(search_text, type = ""){
    
    # type can be : movie, series, episode, game
   
    base_search_url <- "http://www.omdbapi.com/?s="
    type = paste0("type=",type)
    apikey <- "apikey=a79b2c95"
    search <- gsub(" ", "+", search_text)

    search_url <- paste0(base_search_url, paste(search, type, apikey, sep = '&'))
    result <- httr::GET(url = search_url)
    result_httr <- httr::content(result, as = "text")
    result_json <- fromJSON(result_httr)
    
    title <- NA
    type <- NA
    year <- NA
    poster <- NA
    
    for (i in 1:length(result_json$Search$imdbID)){
        imdb_id = result_json$Search$imdbID[i]
        imdb_id_details <- imdb_search(imdb_id)
        title[i] <- imdb_id_details$Title
        type[i] <- imdb_id_details$Type
        year[i] <- imdb_id_details$Year
        poster[i] <- imdb_id_details$Poster
    }
    
    output = data.frame(title, type, suppressWarnings(as.numeric(year)), poster)
    names(output)[1] <- "Title"
    names(output)[2] <- "Type"
    names(output)[3] <- "Year"
    names(output)[4] <- "Poster (link)"
    return (output)
}


In [21]:
get_poster("guardians galaxy")

Title,Type,Year,Poster (link)
<chr>,<chr>,<dbl>,<chr>
Guardians of the Galaxy,movie,2014,https://m.media-amazon.com/images/M/MV5BMTAwMjU5OTgxNjZeQTJeQWpwZ15BbWU4MDUxNDYxODEx._V1_SX300.jpg
Guardians of the Galaxy Vol. 2,movie,2017,https://m.media-amazon.com/images/M/MV5BNjM0NTc0NzItM2FlYS00YzEwLWE0YmUtNTA2ZWIzODc2OTgxXkEyXkFqcGdeQXVyNTgwNzIyNzg@._V1_SX300.jpg
Guardians of the Galaxy,series,NA,https://m.media-amazon.com/images/M/MV5BNDM4NDQxMDU2MV5BMl5BanBnXkFtZTgwMDY2MDQ5NjE@._V1_SX300.jpg
Guardians of the Galaxy: Inferno,movie,2017,https://m.media-amazon.com/images/M/MV5BZGQ0YzEyNWQtNGJiMi00NTAxLThkNDctNGY2ODkzYWMxZmZkXkEyXkFqcGdeQXVyNTAyODkwOQ@@._V1_SX300.jpg
LEGO Marvel Super Heroes - Guardians of the Galaxy: The Thanos Threat,movie,2017,https://m.media-amazon.com/images/M/MV5BMjhlYzVhNTMtMmFkYy00NDhiLTkyNDgtYzhhMTZiMzM2OTA5XkEyXkFqcGdeQXVyNjI2OTgxNzY@._V1_SX300.jpg
Disneyland Resort: Guardians of the Galaxy - Mission Breakout!,movie,2017,https://m.media-amazon.com/images/M/MV5BOTYwZmQwY2MtMzkwMC00ZjllLTg2YWItNzg1MzEzOThjYTkxXkEyXkFqcGdeQXVyMTA1MTY4NTkz._V1_SX300.jpg
Guardians of the Galaxy: The Telltale Series,game,2017,https://m.media-amazon.com/images/M/MV5BMzkwYzJkOTYtOWVlZC00Mzk3LThlZTktYWY3MDM3N2IwZDA3XkEyXkFqcGdeQXVyNTk5Nzg0MDE@._V1_SX300.jpg
Bonus Round: The Making of 'Guardians of the Galaxy Vol. 2',movie,2017,N/A
Guardians of the Galaxy Vol. 3,movie,2023,https://m.media-amazon.com/images/M/MV5BMWM3ZWNlMjgtODZjMi00YTAwLWJhZTktMTM4NTgyMjE0NjMwXkEyXkFqcGdeQXVyNTE1NjY5Mg@@._V1_SX300.jpg


In [24]:
get_poster("guardians galaxy")$`Poster (link)`

[1] "https://m.media-amazon.com/images/M/MV5BMTAwMjU5OTgxNjZeQTJeQWpwZ15BbWU4MDUxNDYxODEx._V1_SX300.jpg"                                
 [2] "https://m.media-amazon.com/images/M/MV5BNjM0NTc0NzItM2FlYS00YzEwLWE0YmUtNTA2ZWIzODc2OTgxXkEyXkFqcGdeQXVyNTgwNzIyNzg@._V1_SX300.jpg"
 [3] "https://m.media-amazon.com/images/M/MV5BNDM4NDQxMDU2MV5BMl5BanBnXkFtZTgwMDY2MDQ5NjE@._V1_SX300.jpg"                                
 [4] "https://m.media-amazon.com/images/M/MV5BZGQ0YzEyNWQtNGJiMi00NTAxLThkNDctNGY2ODkzYWMxZmZkXkEyXkFqcGdeQXVyNTAyODkwOQ@@._V1_SX300.jpg"
 [5] "https://m.media-amazon.com/images/M/MV5BMjhlYzVhNTMtMmFkYy00NDhiLTkyNDgtYzhhMTZiMzM2OTA5XkEyXkFqcGdeQXVyNjI2OTgxNzY@._V1_SX300.jpg"
 [6] "https://m.media-amazon.com/images/M/MV5BOTYwZmQwY2MtMzkwMC00ZjllLTg2YWItNzg1MzEzOThjYTkxXkEyXkFqcGdeQXVyMTA1MTY4NTkz._V1_SX300.jpg"
 [7] "https://m.media-amazon.com/images/M/MV5BMzkwYzJkOTYtOWVlZC00Mzk3LThlZTktYWY3MDM3N2IwZDA3XkEyXkFqcGdeQXVyNTk5Nzg0MDE@._V1_SX300.jpg"
 [8] "N/A"                                                                                                                               
 [9] "https://m.media-amazon.com/images/M/MV5BMWM3ZWNlMjgtODZjMi00YTAwLWJhZTktMTM4NTgyMjE0NjMwXkEyXkFqcGdeQXVyNTE1NjY5Mg@@._V1_SX300.jpg"
[10] "N/A"

In [4]:
# search movies/games/series by search

search <- function(search_text, type = ""){
    
    # type can be : movie, series, episode, game
   
    base_search_url <- "http://www.omdbapi.com/?s="
    type = paste0("type=",type)
    apikey <- "apikey=a79b2c95"
    search <- gsub(" ", "+", search_text)

    #http://www.omdbapi.com/?s=guardians&type=game&apikey=a79b2c95

    search_url <- paste0(base_search_url, paste(search, type, apikey, sep = '&'))
    result <- httr::GET(url = search_url)
    result_httr <- httr::content(result, as = "text")
    result_json <- fromJSON(result_httr)
    
    title <- NA
    type <- NA
    year <- NA
    genre <- NA
    runtime <- NA
    actors <- NA
    director <- NA
    awards <- NA
    rating <- NA
    
    for (i in 1:length(result_json$Search$imdbID)){
        imdb_id = result_json$Search$imdbID[i]
        imdb_id_details <- imdb_search(imdb_id)
        title[i] <- imdb_id_details$Title
        type[i] <- imdb_id_details$Type
        year[i] <- imdb_id_details$Year
        genre[i] <- imdb_id_details$Genre
        runtime[i] <- imdb_id_details$Runtime
        actors[i] <- imdb_id_details$Actors
        director[i] <- imdb_id_details$Director
        awards[i] <- imdb_id_details$Awards
        rating[i] <- imdb_id_details$imdbRating
    }
    
    output = data.frame(title, type, suppressWarnings(as.numeric(year)), genre, suppressWarnings(as.numeric(sapply(strsplit(runtime, " "), "[[", 1))), actors, director, awards, suppressWarnings(as.numeric(rating)))
    names(output)[1] <- "Title"
    names(output)[2] <- "Type"
    names(output)[3] <- "Year"
    names(output)[4] <- "Genre"
    names(output)[5] <- "Runtime (in minutes)"
    names(output)[6] <- "Actors"
    names(output)[7] <- "Director"
    names(output)[8] <- "Awards"
    names(output)[9] <- "IMDB Rating"
    
    return(output) 
}

In [5]:
search("guardians galaxy")

Title,Type,Year,Genre,Runtime (in minutes),Actors,Director,Awards,IMDB Rating
<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
Guardians of the Galaxy,movie,2014,"Action, Adventure, Comedy, Sci-Fi",121,"Chris Pratt, Zoe Saldana, Dave Bautista, Vin Diesel",James Gunn,Nominated for 2 Oscars. Another 52 wins & 100 nominations.,8.0
Guardians of the Galaxy Vol. 2,movie,2017,"Action, Adventure, Comedy, Sci-Fi",136,"Chris Pratt, Zoe Saldana, Dave Bautista, Vin Diesel",James Gunn,Nominated for 1 Oscar. Another 15 wins & 57 nominations.,7.6
Guardians of the Galaxy,series,NA,"Animation, Action, Adventure, Comedy, Drama, Family, Sci-Fi",22,"Kevin Michael Richardson, Vanessa Marshall, David Sobolov, Will Friedle",N/A,2 wins & 2 nominations.,7.4
Guardians of the Galaxy: Inferno,movie,2017,"Short, Comedy, Music",4,"David Hasselhoff, Chris Pratt, Dave Bautista, Karen Gillan","James Gunn, David Yarovesky",N/A,8.2
LEGO Marvel Super Heroes - Guardians of the Galaxy: The Thanos Threat,movie,2017,"Animation, Short, Action, Adventure, Comedy, Fantasy, Sci-Fi",22,"Jonathan Adams, Trevor Devall, Will Friedle, Jennifer Hale",Michael D. Black,N/A,6.0
Disneyland Resort: Guardians of the Galaxy - Mission Breakout!,movie,2017,"Short, Action, Adventure, Comedy, Fantasy, Sci-Fi",21,"Bradley Cooper, Chris Pratt, Zoe Saldana, Dave Bautista",N/A,1 nomination.,8.3
Guardians of the Galaxy: The Telltale Series,game,2017,"Adventure, Sci-Fi",NA,"Scott Porter, Brandon Paul Eells, Emily O'Brien, Adam Harrington","Jonathan Stauder, Sean Ainsworth",N/A,8.8
Bonus Round: The Making of 'Guardians of the Galaxy Vol. 2',movie,2017,"Documentary, Short",39,"Kevin Feige, James Gunn, Chris Pratt",N/A,N/A,7.1
Guardians of the Galaxy Vol. 3,movie,2023,"Action, Adventure, Comedy, Sci-Fi, Thriller",NA,"Elizabeth Debicki, Karen Gillan, Zoe Saldana, Chris Pratt",James Gunn,N/A,NA


In [26]:
# get recommendations by Genre based on popularity

get_top10_recommendations <- function(genre = 'adventure'){
    
    # Genre can be : action, adventure, comedy, drama, scifi, horror, animation
    
    action <- c('tt9140560', 'tt2560140', 'tt8179402', 'tt2306299', 'tt2531336', 'tt10332588', 'tt7221388', 'tt8111088', 'tt0944947', 'tt6156584')    
    adventure <- c('tt2560140', 'tt8179402', 'tt2306299', 'tt10332588', 'tt8111088', 'tt0944947', 'tt9845564', 'tt1831804', 'tt2948372', 'tt0293429')
    comedy <- c('tt9140560', 'tt4477976', 'tt10332588', 'tt7221388', 'tt9140342', 'tt8690918', 'tt0386676', 'tt3526078', 'tt2948372', 'tt1586680')
    drama <- c('tt9140560', 'tt3661210', 'tt10016180', 'tt2560140', 'tt8740790', 'tt9012876', 'tt8179402', 'tt6857376', 'tt3230854', 'tt2306299')
    scifi <- c('tt9140560', 'tt3230854', 'tt8111088', 'tt6156584', 'tt5034838', 'tt8690918', 'tt1831804', 'tt10333426', 'tt6723592', 'tt1190634')
    horror <- c('tt2560140', 'tt1831804', 'tt1520211', 'tt0460681', 'tt4574334', 'tt12664876', 'tt1844624', 'tt1405406', 'tt7557108', 'tt8068860')
    animation <- c('tt2560140', 'tt2948372', 'tt0458290', 'tt0096697', 'tt5363918', 'tt5109280', 'tt2861424', 'tt0182576', 'tt9335498', 'tt0121955')

    top10 <- NA
    if (genre == 'action') {
        top10 = action
        } else if ( genre == 'comedy') {
        top10 = comedy
        } else if ( genre == 'drama') {
        top10 = drama
        } else if ( genre == 'scifi') {
        top10 = scifi
        } else if ( genre == 'horror') {
        top10 = horror
        } else if ( genre == 'animation'){
        top10 = animation
        } else {
        top10 = adventure
        }
    
    base_search_url <- "http://www.omdbapi.com/?s="
    apikey <- "apikey=a79b2c95"
    
    title <- NA
    type <- NA
    year <- NA
    genre <- NA
    runtime <- NA
    actors <- NA
    director <- NA
    awards <- NA
    rating <- NA
    
    for (i in 1:length(top10)){
        imdb_id = top10[i]
        imdb_id_details <- imdb_search(imdb_id)
        title[i] <- imdb_id_details$Title
        type[i] <- imdb_id_details$Type
        year[i] <- imdb_id_details$Year
        genre[i] <- imdb_id_details$Genre
        runtime[i] <- imdb_id_details$Runtime
        actors[i] <- imdb_id_details$Actors
        director[i] <- imdb_id_details$Director
        awards[i] <- imdb_id_details$Awards
        rating[i] <- imdb_id_details$imdbRating
    }
    
    output = data.frame(title, type, suppressWarnings(as.numeric(year)), genre, suppressWarnings(as.numeric(sapply(strsplit(runtime, " "), "[[", 1))), actors, director, awards, suppressWarnings(as.numeric(rating)))
    names(output)[1] <- "Title"
    names(output)[2] <- "Type"
    names(output)[3] <- "Year"
    names(output)[4] <- "Genre"
    names(output)[5] <- "Runtime (in minutes)"
    names(output)[6] <- "Actors"
    names(output)[7] <- "Director"
    names(output)[8] <- "Awards"
    names(output)[9] <- "IMDB Rating"
    
    return(output) 
}

In [25]:
get_top10_recommendations()

Title,Type,Year,Genre,Runtime (in minutes),Actors,Director,Awards,IMDB Rating
<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
Attack on Titan,series,NA,"Animation, Action, Adventure, Drama, Fantasy, Horror",24,"Marina Inoue, Yûki Kaji, Yui Ishikawa, Josh Grelle",N/A,8 wins & 7 nominations.,8.8
Fate: The Winx Saga,series,NA,"Action, Adventure, Drama, Fantasy",NA,"Abigail Cowen, Danny Griffin, Hannah van der Westhuysen, Elisha Applebaum",N/A,N/A,NA
Vikings,series,NA,"Action, Adventure, Drama, History, Romance, War",44,"Katheryn Winnick, Gustaf Skarsgård, Alexander Ludwig, Georgia Hirst",N/A,Won 1 Primetime Emmy. Another 38 wins & 121 nominations.,8.5
Finding 'Ohana,movie,2021,"Action, Adventure, Comedy, Family",123,"Kelly Hu, Ke Huy Quan, Chris Parnell, Marc Evan Jackson",Jude Weng,N/A,NA
The Mandalorian,series,NA,"Action, Adventure, Sci-Fi",40,Pedro Pascal,N/A,Won 7 Primetime Emmys. Another 15 wins & 30 nominations.,8.8
Game of Thrones,series,NA,"Action, Adventure, Drama, Fantasy, Romance",57,"Peter Dinklage, Lena Headey, Emilia Clarke, Kit Harington",N/A,Won 1 Golden Globe. Another 374 wins & 602 nominations.,9.3
Below Zero,movie,2021,"Action, Adventure, Crime, Thriller",106,"Javier Gutiérrez, Àlex Monner, Édgar Vittorino, Patrick Criado",Lluís Quílez,N/A,NA
The Stand,series,NA,"Adventure, Drama, Fantasy, Horror, Sci-Fi, Thriller",NA,"James Marsden, Odessa Young, Gordon Cormier, Amber Heard",N/A,N/A,5.5
Soul,movie,2020,"Animation, Adventure, Comedy, Family, Fantasy, Music",100,"Jamie Foxx, Tina Fey, Graham Norton, Rachel House","Pete Docter, Kemp Powers(co-director)",37 wins & 28 nominations.,8.1


In [29]:
sample(get_top10_recommendations('drama')[[4]], 1)

[1] "Animation, Action, Adventure, Drama, Fantasy, Horror"

In [26]:
get_top10_recommendations('animation')

Title,Type,Year,Genre,Runtime (in minutes),Actors,Director,Awards,IMDB Rating
<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
Attack on Titan,series,NA,"Animation, Action, Adventure, Drama, Fantasy, Horror",24,"Marina Inoue, Yûki Kaji, Yui Ishikawa, Josh Grelle",N/A,8 wins & 7 nominations.,8.8
Soul,movie,2020,"Animation, Adventure, Comedy, Family, Fantasy, Music",100,"Jamie Foxx, Tina Fey, Graham Norton, Rachel House","Pete Docter, Kemp Powers(co-director)",37 wins & 28 nominations.,8.1
Star Wars: The Clone Wars,series,NA,"Animation, Action, Adventure, Drama, Fantasy, Sci-Fi",23,"Tom Kane, Dee Bradley Baker, Matt Lanter, James Arnold Taylor",N/A,23 wins & 69 nominations.,8.3
The Simpsons,series,NA,"Animation, Comedy",22,"Dan Castellaneta, Nancy Cartwright, Harry Shearer, Julie Kavner",N/A,Nominated for 1 Golden Globe. Another 168 wins & 302 nominations.,8.7
Disenchantment,series,NA,"Animation, Action, Adventure, Comedy, Fantasy",30,"Abbi Jacobson, Eric André, Nat Faxon, John DiMaggio",N/A,N/A,7.2
Raya and the Last Dragon,movie,2021,"Animation, Action, Adventure, Comedy, Family, Fantasy, Musical",NA,"Kelly Marie Tran, Awkwafina","Don Hall, Carlos López Estrada, Paul Briggs(co-director), John Ripa(co-director)",N/A,NA
Rick and Morty,series,NA,"Animation, Adventure, Comedy, Sci-Fi",23,"Justin Roiland, Chris Parnell, Spencer Grammer, Sarah Chalke",N/A,Won 2 Primetime Emmys. Another 15 wins & 21 nominations.,9.2
Family Guy,series,NA,"Animation, Comedy",22,"Seth MacFarlane, Alex Borstein, Seth Green, Mila Kunis",N/A,Won 9 Primetime Emmys. Another 28 wins & 101 nominations.,8.1
Demon Slayer: Kimetsu No Yaiba,series,NA,"Animation, Action, Fantasy, Thriller",25,"Natsuki Hanae, Zach Aguilar, Abby Trott, Aleks Le",N/A,1 win & 1 nomination.,8.7


In [15]:
surprise_me <- function(genre = 'adventure'){
    
    # Genre can be : action, adventure, comedy, drama, scifi, horror, animation
    
    action <- c('tt9140560', 'tt2560140', 'tt8179402', 'tt2306299', 'tt2531336', 'tt10332588', 'tt7221388', 'tt8111088', 'tt0944947', 'tt6156584')    
    adventure <- c('tt2560140', 'tt8179402', 'tt2306299', 'tt10332588', 'tt8111088', 'tt0944947', 'tt9845564', 'tt1831804', 'tt2948372', 'tt0293429')
    comedy <- c('tt9140560', 'tt4477976', 'tt10332588', 'tt7221388', 'tt9140342', 'tt8690918', 'tt0386676', 'tt3526078', 'tt2948372', 'tt1586680')
    drama <- c('tt9140560', 'tt3661210', 'tt10016180', 'tt2560140', 'tt8740790', 'tt9012876', 'tt8179402', 'tt6857376', 'tt3230854', 'tt2306299')
    scifi <- c('tt9140560', 'tt3230854', 'tt8111088', 'tt6156584', 'tt5034838', 'tt8690918', 'tt1831804', 'tt10333426', 'tt6723592', 'tt1190634')
    horror <- c('tt2560140', 'tt1831804', 'tt1520211', 'tt0460681', 'tt4574334', 'tt12664876', 'tt1844624', 'tt1405406', 'tt7557108', 'tt8068860')
    animation <- c('tt2560140', 'tt2948372', 'tt0458290', 'tt0096697', 'tt5363918', 'tt5109280', 'tt2861424', 'tt0182576', 'tt9335498', 'tt0121955')
    recommendations <- c(sample(action[1:5], 1), sample(adventure[1:5], 1), sample(comedy[1:5], 1), sample(drama[1:5], 1), sample(scifi[1:5], 1), sample(horror[1:5], 1), sample(animation[1:5], 1))
        
    base_search_url <- "http://www.omdbapi.com/?s="
    apikey <- "apikey=a79b2c95"
    
    title <- NA
    type <- NA
    year <- NA
    genre <- NA
    runtime <- NA
    actors <- NA
    director <- NA
    awards <- NA
    rating <- NA
    
    for (i in 1:length(recommendations)){
        imdb_id = recommendations[i]
        imdb_id_details <- imdb_search(imdb_id)
        title[i] <- imdb_id_details$Title
        type[i] <- imdb_id_details$Type
        year[i] <- imdb_id_details$Year
        genre[i] <- imdb_id_details$Genre
        runtime[i] <- imdb_id_details$Runtime
        actors[i] <- imdb_id_details$Actors
        director[i] <- imdb_id_details$Director
        awards[i] <- imdb_id_details$Awards
        rating[i] <- imdb_id_details$imdbRating
    }
    
    output = data.frame(title, type, suppressWarnings(as.numeric(year)), genre, suppressWarnings(as.numeric(sapply(strsplit(runtime, " "), "[[", 1))), actors, director, awards, suppressWarnings(as.numeric(rating)))
    names(output)[1] <- "Title"
    names(output)[2] <- "Type"
    names(output)[3] <- "Year"
    names(output)[4] <- "Genre"
    names(output)[5] <- "Runtime (in minutes)"
    names(output)[6] <- "Actors"
    names(output)[7] <- "Director"
    names(output)[8] <- "Awards"
    names(output)[9] <- "IMDB Rating"
    
    return(output) 
}

In [16]:
surprise_me()

Title,Type,Year,Genre,Runtime (in minutes),Actors,Director,Awards,IMDB Rating
<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
Attack on Titan,series,NA,"Animation, Action, Adventure, Drama, Fantasy, Horror",24,"Marina Inoue, Yûki Kaji, Yui Ishikawa, Josh Grelle",N/A,8 wins & 7 nominations.,8.8
Finding 'Ohana,movie,2021,"Action, Adventure, Comedy, Family",123,"Kelly Hu, Ke Huy Quan, Chris Parnell, Marc Evan Jackson",Jude Weng,N/A,NA
Finding 'Ohana,movie,2021,"Action, Adventure, Comedy, Family",123,"Kelly Hu, Ke Huy Quan, Chris Parnell, Marc Evan Jackson",Jude Weng,N/A,NA
Bridgerton,series,NA,"Drama, Romance",60,"Jonathan Bailey, Ruby Barker, Harriet Cains, Bessie Carter",N/A,N/A,7.4
Snowpiercer,series,NA,"Action, Drama, Sci-Fi, Thriller",60,"Jennifer Connelly, Daveed Diggs, Mickey Sumner, Alison Wright",N/A,2 wins & 4 nominations.,6.7
Attack on Titan,series,NA,"Animation, Action, Adventure, Drama, Fantasy, Horror",24,"Marina Inoue, Yûki Kaji, Yui Ishikawa, Josh Grelle",N/A,8 wins & 7 nominations.,8.8
Attack on Titan,series,NA,"Animation, Action, Adventure, Drama, Fantasy, Horror",24,"Marina Inoue, Yûki Kaji, Yui Ishikawa, Josh Grelle",N/A,8 wins & 7 nominations.,8.8


In [11]:
surprise_me_new <- function(top = 1){
    
    # Genre can be : action, adventure, comedy, drama, scifi, horror, animation
    
    if (top > 10 | top < 1) 
    {
    stop(paste0("Top number must beween 1 and 10"))
    }
    
    action <- c('tt9140560', 'tt2560140', 'tt8179402', 'tt2306299', 'tt2531336', 'tt10332588', 'tt7221388', 'tt8111088', 'tt0944947', 'tt6156584')    
    adventure <- c('tt2560140', 'tt8179402', 'tt2306299', 'tt10332588', 'tt8111088', 'tt0944947', 'tt9845564', 'tt1831804', 'tt2948372', 'tt0293429')
    comedy <- c('tt9140560', 'tt4477976', 'tt10332588', 'tt7221388', 'tt9140342', 'tt8690918', 'tt0386676', 'tt3526078', 'tt2948372', 'tt1586680')
    drama <- c('tt9140560', 'tt3661210', 'tt10016180', 'tt2560140', 'tt8740790', 'tt9012876', 'tt8179402', 'tt6857376', 'tt3230854', 'tt2306299')
    scifi <- c('tt9140560', 'tt3230854', 'tt8111088', 'tt6156584', 'tt5034838', 'tt8690918', 'tt1831804', 'tt10333426', 'tt6723592', 'tt1190634')
    horror <- c('tt2560140', 'tt1831804', 'tt1520211', 'tt0460681', 'tt4574334', 'tt12664876', 'tt1844624', 'tt1405406', 'tt7557108', 'tt8068860')
    animation <- c('tt2560140', 'tt2948372', 'tt0458290', 'tt0096697', 'tt5363918', 'tt5109280', 'tt2861424', 'tt0182576', 'tt9335498', 'tt0121955')
    recommendations <- c(sample(action[1:10], top), sample(adventure[1:10], top), sample(comedy[1:10], top), sample(drama[1:10], top), sample(scifi[1:10], top), sample(horror[1:10], top), sample(animation[1:10], top))
        
    base_search_url <- "http://www.omdbapi.com/?s="
    apikey <- "apikey=a79b2c95"
    
    title <- NA
    type <- NA
    year <- NA
    genre <- NA
    runtime <- NA
    actors <- NA
    director <- NA
    awards <- NA
    rating <- NA
    
    for (i in 1:length(recommendations)){
        imdb_id = recommendations[i]
        imdb_id_details <- imdb_search(imdb_id)
        title[i] <- imdb_id_details$Title
        type[i] <- imdb_id_details$Type
        year[i] <- imdb_id_details$Year
        genre[i] <- imdb_id_details$Genre
        runtime[i] <- imdb_id_details$Runtime
        actors[i] <- imdb_id_details$Actors
        director[i] <- imdb_id_details$Director
        awards[i] <- imdb_id_details$Awards
        rating[i] <- imdb_id_details$imdbRating
    }
    
    output = data.frame(title, type, suppressWarnings(as.numeric(year)), genre, suppressWarnings(as.numeric(sapply(strsplit(runtime, " "), "[[", 1))), actors, director, awards, suppressWarnings(as.numeric(rating)))
    names(output)[1] <- "Title"
    names(output)[2] <- "Type"
    names(output)[3] <- "Year"
    names(output)[4] <- "Genre"
    names(output)[5] <- "Runtime (in minutes)"
    names(output)[6] <- "Actors"
    names(output)[7] <- "Director"
    names(output)[8] <- "Awards"
    names(output)[9] <- "IMDB Rating"
    
    return(output) 
}

In [12]:
surprise_me_new()

Title,Type,Year,Genre,Runtime (in minutes),Actors,Director,Awards,IMDB Rating
<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>
The Mandalorian,series,NA,"Action, Adventure, Sci-Fi",40,Pedro Pascal,N/A,Won 7 Primetime Emmys. Another 15 wins & 30 nominations.,8.8
The Mandalorian,series,NA,"Action, Adventure, Sci-Fi",40,Pedro Pascal,N/A,Won 7 Primetime Emmys. Another 15 wins & 30 nominations.,8.8
Resident Alien,series,NA,"Comedy, Drama, Mystery, Sci-Fi",NA,"Elizabeth Bowen, Levi Fiehler, Meredith Garretson, Corey Reynolds",N/A,N/A,NA
Fate: The Winx Saga,series,NA,"Action, Adventure, Drama, Fantasy",NA,"Abigail Cowen, Danny Griffin, Hannah van der Westhuysen, Elisha Applebaum",N/A,N/A,NA
The Stand,series,NA,"Adventure, Drama, Fantasy, Horror, Sci-Fi, Thriller",NA,"James Marsden, Odessa Young, Gordon Cormier, Amber Heard",N/A,N/A,5.5
The Stand,series,NA,"Adventure, Drama, Fantasy, Horror, Sci-Fi, Thriller",NA,"James Marsden, Odessa Young, Gordon Cormier, Amber Heard",N/A,N/A,5.5
Attack on Titan,series,NA,"Animation, Action, Adventure, Drama, Fantasy, Horror",24,"Marina Inoue, Yûki Kaji, Yui Ishikawa, Josh Grelle",N/A,8 wins & 7 nominations.,8.8


In [13]:
surprise_me_new(12)

ERROR: Error in surprise_me_new(12): Top number must beween 1 and 10


In [14]:
surprise_me_new(-1)

ERROR: Error in surprise_me_new(-1): Top number must beween 1 and 10
